# Find Christoffel symbols using sympy

In [1]:
import numpy as np
from sympy import *
from tqdm import trange
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
init_printing()
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial
def flatten(l):
    return [item for sublist in l for item in sublist]
def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list
def get_gamma_abc(rho,mu,nu,gamma,indices):
    ind = -1
    for i in range(len(indices)):
        if '(%d_%d_%d)'%(rho,mu,nu) in indices[i]:
            ind = i; break
    if ind==-1: return 0
    else: return gamma[ind]
def dm(i,j,k):
    return diff(metric[i][j],coor[k])
def gamma(i,j,k):
    s = 0
    for l in range(4):
        s+=0.5*metricI[i][l]*(dm(k,l,j)+dm(l,j,k)-dm(j,k,l))
    s = s.subs(subdict)
    # exploit that max quadratic degree
    nexpand = len(expandlist)
    for v in range(nexpand):
        for w in range(v):
            s = Taylor_polynomial_sympy(s,[expandlist[v],expandlist[w]],
                                        [originlist[v],originlist[w]],order)
    return simplify(nsimplify((s))).expand()

In [2]:
######## MENU ##################
conformaltime = 1
metricchoice  = dict({1:'flrw',2:'psiphi',
                      3:'wB',4:'full'})[2]
order = 1
################################

In [3]:
# initialise
foundInverse=(metricchoice!='wB' and metricchoice!='full')
t,x,y,z = symbols("t x y z")
coor = [t,x,y,z]
psi,phi,aH, a = symbols('Psi Phi \mathcal{H} a') # not functions
af = Function('a',real=True)
psif = Function("Psi",real=True)
phif = Function("Phi",real=True)
dkpsif,dkpsi = [],[]
dkphif,dkphi = [],[]
for i in range(4):
    dkphi.append(symbols('\partial_{%s}\Phi'%coor[i]))
    dkpsi.append(symbols('\partial_{%s}\Psi'%coor[i]))
    dkpsif.append(psif(*coor).diff(coor[i]))
    dkphif.append(phif(*coor).diff(coor[i]))
# fix vector and tensor
Bi = symbols('B1 B2 B3')
Bif= []
dBi= [[],[],[]]
dkBif=[[],[],[]]
dkBi =[[],[],[]]
hij = []
hijf=[[],[],[]]
dkhij =[[[],[],[]],[[],[],[]],[[],[],[]]]
dkhijf=[[[],[],[]],[[],[],[]],[[],[],[]]]
for i in range(1,4):
    hij += [list(symbols('h%d1 h%d2 h%d3'%(i,i,i)))]
    for j in range(1,4):
        if (j>=i):
            hijf[i-1]+=[Function("h_%d%d"%(i,j),real=True)]
        else:
            hijf[i-1]+=[Function("h_%d%d"%(j,i),real=True)]
        for k in range(1,5):
            if (j>=i):
                dkhijf[i-1][j-1].append(hijf[i-1][j-1](*coor).diff(coor[k-1]))
                dkhij[i-1][j-1].append(symbols('\partial_%sh_{%d%d}' % (coor[k-1],i,j)))
            else:
                dkhijf[i-1][j-1].append(hijf[j-1][i-1](*coor).diff(coor[k-1]))
                dkhij[i-1][j-1].append(symbols('\partial_%sh_{%d%d}' % (coor[k-1],j,i)))
    Bif += [Function("B_%d"%i,real=True)]
    for k in range(1,5):
        dkBif[i-1].append(Bif[i-1](*coor).diff(coor[k-1]))
        dkBi[i-1].append(symbols('\partial_%sB_{%d}'%(coor[k-1],i)))
                  
expandlist = [phi,psi,*dkphi,*dkpsi]
if (not foundInverse): expandlist+=[*Bi,*flatten(dkBi)]
if (metricchoice=='full'): expandlist+=flatten(hij)
if (metricchoice=='full'): expandlist+=flatten(flatten(dkhij))
expandlist = unique(expandlist)
originlist =  [0]*len(expandlist)
subdict = {psif(*coor):psi,phif(*coor):phi, af(t):a}
if (conformaltime>0):
    subdict[af(t).diff(t)] = aH*a
else:
    subdict[af(t).diff(t)] = aH
sflist = [*dkpsif,*dkphif,*flatten(dkBif),*flatten(flatten(dkhijf))]
slist  = [*dkpsi ,*dkphi ,*flatten(dkBi ),*flatten(flatten(dkhij ))]
for i in range(3):
    subdict[Bif[i](*coor)] = Bi[i]
    for j in range(3):
        subdict[hijf[i][j](*coor)] = hij[i][j]
for i in range(len(sflist)):
    subdict[sflist[i]] = slist[i]

if (metricchoice=='full'):
    metric = [[af(t)**(2*conformaltime)*(-1-2*psif(*coor)),-af(t)**(1+conformaltime)*Bif[0](*coor),-af(t)**(1+conformaltime)*Bif[1](*coor),-af(t)**(1+conformaltime)*Bif[2](*coor)],
                [-af(t)**(1+conformaltime)*Bif[0](*coor),af(t)**2*(1-2*phif(*coor)+hijf[0][0](*coor)),af(t)**2*hijf[0][1](*coor),af(t)**2*hijf[0][2](*coor)],
                [-af(t)**(1+conformaltime)*Bif[1](*coor),af(t)**2*hijf[1][0](*coor),af(t)**2*(1-2*phif(*coor)+hijf[1][1](*coor)),af(t)**2*hijf[1][2](*coor)],
                [-af(t)**(1+conformaltime)*Bif[2](*coor),af(t)**2*hijf[2][0](*coor),af(t)**2*hijf[2][1](*coor),af(t)**2*(1-2*phif(*coor)+hijf[2][2](*coor))]]
    metricI = [[af(t)**(-2*conformaltime)*(-1+2*psif(*coor)),-af(t)**(-1-conformaltime)*Bif[0](*coor),-af(t)**(-1-conformaltime)*Bif[1](*coor),-af(t)**(-1-conformaltime)*Bif[2](*coor)],
                [-af(t)**(-1-conformaltime)*Bif[0](*coor),af(t)**-2*(1+2*phif(*coor)-hijf[0][0](*coor)),-af(t)**-2*hijf[0][1](*coor),-af(t)**-2*hijf[0][2](*coor)],
                [-af(t)**(-1-conformaltime)*Bif[1](*coor),-af(t)**-2*hijf[1][0](*coor),af(t)**-2*(1+2*phif(*coor)-hijf[1][1](*coor)),-af(t)**-2*hijf[1][2](*coor)],
                [-af(t)**(-1-conformaltime)*Bif[2](*coor),-af(t)**-2*hijf[2][0](*coor),-af(t)**-2*hijf[2][1](*coor),af(t)**-2*(1+2*phif(*coor)-hijf[2][2](*coor))]]
elif (metricchoice=='wB'):
    metric = [[af(t)**(2*conformaltime)*(-1-2*psif(*coor)),-af(t)**(1+conformaltime)*Bif[0](*coor),-af(t)**(1+conformaltime)*Bif[1](*coor),-af(t)**(1+conformaltime)*Bif[2](*coor)],
                [-af(t)**(1+conformaltime)*Bif[0](*coor),af(t)**2*(1-2*phif(*coor)),0,0],
                [-af(t)**(1+conformaltime)*Bif[1](*coor),0,af(t)**2*(1-2*phif(*coor)),0],
                [-af(t)**(1+conformaltime)*Bif[2](*coor),0,0,af(t)**2*(1-2*phif(*coor))]]
    metricI= [[af(t)**(-2*conformaltime)*(-1+2*psif(*coor)),-af(t)**(-1-conformaltime)*Bif[0](*coor),-af(t)**(-1-conformaltime)*Bif[1](*coor),-af(t)**(-1-conformaltime)*Bif[2](*coor)],
                [-af(t)**(-1-conformaltime)*Bif[0](*coor),af(t)**-2*(1+2*phif(*coor)),0,0],
                [-af(t)**(-1-conformaltime)*Bif[1](*coor),0,af(t)**-2*(1+2*phif(*coor)),0],
                [-af(t)**(-1-conformaltime)*Bif[2](*coor),0,0,af(t)**-2*(1+2*phif(*coor))]]
elif (metricchoice=='psiphi'):
    metric = [[af(t)**(2*conformaltime)*(-1-2*psif(*coor)),0,0,0],
                [0,af(t)**2*(1-2*phif(*coor)),0,0],
                [0,0,af(t)**2*(1-2*phif(*coor)),0],
                [0,0,0,af(t)**2*(1-2*phif(*coor))]]
    metricI= [[af(t)**(-2*conformaltime)*(-1+2*psif(*coor)),0,0,0],
                [0,af(t)**-2*(1+2*phif(*coor)),0,0],
                [0,0,af(t)**-2*(1+2*phif(*coor)),0],
                [0,0,0,af(t)**-2*(1+2*phif(*coor))]]
elif (metricchoice=='flrw'):
    metric = [[-af(t)**(2*conformaltime),0,0,0],
                [0,af(t)**2,0,0],
                [0,0,af(t)**2,0],
                [0,0,0,af(t)**2]]

In [4]:
# find christoffel symbols
chris = []
chris_inds = []

for ac in trange(4):
    for b in range(4):
        for c in range(4):
            res = gamma(ac,b,c)
            if res==0: pass
            else:
                chris.append(res)     
                chris_inds.append(["(%d_%d_%d)"%(ac,b,c)])   

100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.12it/s]


In [22]:
# check result and save
#get_gamma_abc(0,1,1,chris,chris_inds)
# find the traces
trace0 = 0
trace1 = 0
cntr0  = 0
for i in range(0,4):
    trace0 += get_gamma_abc(i,0,i,chris,chris_inds)
    trace1 += get_gamma_abc(i,1,i,chris,chris_inds)
    for j in range(4):
        cntr0+=metricI[i][j]*get_gamma_abc(0,i,j,chris,chris_inds)
cntr0 = cntr0.subs(subdict)
trace0#.expand()
trace1#.expand()
#cntr0
if False:
    if (conformaltime==0): np.save('christoffel_cosmic_full',np.asanyarray([chris,chris_inds],dtype=object))
    else: np.save('christoffel_conformal_full',np.asanyarray([chris,chris_inds],dtype=object))

In [24]:
chris, chris_inds = np.load('christoffel_conformal_full.npy',allow_pickle=True)
get_gamma_abc(1,0,1,chris,chris_inds)
get_gamma_abc(2,0,2,chris,chris_inds)
get_gamma_abc(1,0,3,chris,chris_inds)